## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-29-06-54-00 +0000,wsj,Swiss Asset Manager GAM Opposes Takeover of Ho...,https://www.wsj.com/business/deals/swiss-asset...
1,2025-12-29-06-45-01 +0000,bbc,Fire in nursing home kills 16 in Indonesia,https://www.bbc.com/news/articles/cde6e941xdjo...
2,2025-12-29-06-14-08 +0000,nypost,Wildlife influencer ‘happy to be alive’ after ...,https://nypost.com/2025/12/29/us-news/influenc...
3,2025-12-29-06-05-53 +0000,bbc,'There are many challenges': Syrians in Turkey...,https://www.bbc.com/news/articles/cd74pxvrp10o...
4,2025-12-29-06-01-10 +0000,missionlocal,"11,000 lose power in yet another S.F. outage i...",https://missionlocal.org/2025/12/sf-power-outa...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2432/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
166,trump,20
216,new,10
400,brigitte,8
401,bardot,8
180,zelensky,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
94,2025-12-28-19-01-49 +0000,nypost,Trump believes ‘we have makings of a deal’ to ...,https://nypost.com/2025/12/28/world-news/trump...,62
86,2025-12-28-20-02-25 +0000,nypost,Putin’s top negotiator posts video of himself ...,https://nypost.com/2025/12/28/world-news/putin...,59
102,2025-12-28-17-32-52 +0000,nyt,Zelensky to Meet With Trump at Mar-a-Lago Abou...,https://www.nytimes.com/2025/12/28/world/europ...,52
115,2025-12-28-15-51-54 +0000,nypost,Trump touts ‘good and very productive’ call wi...,https://nypost.com/2025/12/28/us-news/zelensky...,49
50,2025-12-29-00-54-00 +0000,wsj,Trump Says Talks to End Ukraine War Yield Prog...,https://www.wsj.com/world/trump-zelensky-russi...,48


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
94,62,2025-12-28-19-01-49 +0000,nypost,Trump believes ‘we have makings of a deal’ to ...,https://nypost.com/2025/12/28/world-news/trump...
126,39,2025-12-28-14-17-31 +0000,cbc,"Brigitte Bardot, icon of French cinema turned ...",https://www.cbc.ca/news/entertainment/brigitte...
7,32,2025-12-29-05-52-42 +0000,nypost,Minnesota Gov. Tim Walz on the defensive as fr...,https://nypost.com/2025/12/29/us-news/minnesot...
107,27,2025-12-28-17-00-00 +0000,wsj,The U.S. Air Force’s​ new Wedgetail ​jet progr...,https://www.wsj.com/politics/national-security...
27,24,2025-12-29-03-05-04 +0000,nypost,77-year-old gunman killed after opening fire a...,https://nypost.com/2025/12/28/us-news/77-year-...
45,23,2025-12-29-01-26-11 +0000,nypost,Heart-pounding video captures moment cop stops...,https://nypost.com/2025/12/28/us-news/heart-po...
125,21,2025-12-28-14-17-54 +0000,nypost,Hochul throws weight behind Mamdani’s multi-bi...,https://nypost.com/2025/12/28/us-news/gov-hoch...
86,20,2025-12-28-20-02-25 +0000,nypost,Putin’s top negotiator posts video of himself ...,https://nypost.com/2025/12/28/world-news/putin...
52,20,2025-12-29-00-43-58 +0000,nypost,DC pipe bomb suspect Brian Cole Jr.’s possible...,https://nypost.com/2025/12/28/us-news/dc-pipe-...
112,20,2025-12-28-16-00-00 +0000,wsj,"As Memories of Fukushima Fade, Japan Seeks Big...",https://www.wsj.com/world/asia/as-memories-of-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
